In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torch.optim as optim
from torch import nn

# Choose User Path

In [3]:
from enum import Enum
class Pathes(Enum):
  HADAR = 1
  RAN = 2
  ITAMAR = 3

In [4]:
import sys

##Just choose here the user
path = Pathes.ITAMAR

if(path == Pathes.HADAR):
  PATH='/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_Combined/Weights.pt'
  sys.path.insert(0,'/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_Combined')
  train_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/train.txt'
  test_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/test.txt'
  w2v_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/glove.6B.100d.txt'
elif(path == Pathes.RAN):
  PATH='/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_Combined/Weights.pt'
  sys.path.insert(0,'/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_Combined')
  train_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/train.txt'
  test_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/test.txt'
  w2v_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/glove.6B.100d.txt'
elif(path == Pathes.ITAMAR):
  PATH='/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_Combined/Weights.pt'
  sys.path.insert(0,'/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_Combined')
  #train_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/train.txt'
  train_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/new_train.txt'
  test_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/test.txt'
  val_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/validation.txt'
  w2v_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/glove.6B.100d.txt'

# Train SVB/Classifier on all the other models output

In [9]:
import numpy as np
from numpy import asarray

In [10]:
%cd '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_Combined/ModelOutputData'
Y_Seq2Seq_GroundProof_Train = np.load('Y_Seq2Seq_GroundProof_Train.npy')
Y_RNN_GroundProof_Train = np.load('Y_RNN_GroundProof_Train.npy')
Y_Seq2Seq_GroundProof_Test = np.load('Y_Seq2Seq_GroundProof_Test.npy')
Y_RNN_GroundProof_Test = np.load('Y_RNN_GroundProof_Test.npy')

/content/gdrive/.shortcut-targets-by-id/1XNTZrFPo5LZhTNZ3l5LoLnleAlL4h-1v/IntroToDeep/Project/codes/Model_Combined/ModelOutputData


In [16]:
%cd '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_Combined/ModelOutputData'
Y_Seq2Seq_Pred_Train = np.load('Y_Seq2Seq_Pred_Train.npy')
Y_RNN_Pred_Train = np.load('Y_RNN_Pred_Train.npy')
Y_Seq2Seq_Pred_Test = np.load('Y_Seq2Seq_Pred_Test.npy')
Y_RNN_Pred_Test = np.load('Y_RNN_Pred_Test.npy')

/content/gdrive/.shortcut-targets-by-id/1XNTZrFPo5LZhTNZ3l5LoLnleAlL4h-1v/IntroToDeep/Project/codes/Model_Combined/ModelOutputData


In [23]:
%cd '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_Combined/ModelOutputData'
Y_RCNN_GroundProof_Train = np.load('Y_RCNN_GroundProof_Train.npy')
Y_RCNN_GroundProof_Test = np.load('Y_RCNN_GroundProof_Test.npy')
Y_RCNN_Pred_Train = np.load('Y_RCNN_Pred_Train.npy')

/content/gdrive/.shortcut-targets-by-id/1XNTZrFPo5LZhTNZ3l5LoLnleAlL4h-1v/IntroToDeep/Project/codes/Model_Combined/ModelOutputData


In [34]:
%cd '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_Combined/ModelOutputData'
Y_CNN_GroundProof_Train = np.load('Y_CNN_GroundProof_Train.npy')
Y_CNN_GroundProof_Test = np.load('Y_CNN_GroundProof_Test.npy')
Y_CNN_Pred_Train = np.load('Y_CNN_Pred_Train.npy')

/content/gdrive/.shortcut-targets-by-id/1XNTZrFPo5LZhTNZ3l5LoLnleAlL4h-1v/IntroToDeep/Project/codes/Model_Combined/ModelOutputData


# Create X and Y tensors


In [22]:
###Y_Label
Y_label = Y_RCNN_GroundProof_Train
Y_label.shape

(128000,)

In [38]:
#Y_RNN_Pred_Train
#Y_Seq2Seq_Pred_Train
#Y_RCNN_Pred_Train
#Y_CNN_GroundProof_Train
X_Vector = np.stack((Y_RNN_Pred_Train, Y_Seq2Seq_Pred_Train, Y_RCNN_Pred_Train, Y_CNN_Pred_Train))
X_Vector.shape

(4, 128000)

In [39]:
X = X_Vector.transpose()
X.shape
i=5
print(X[i])

[1 1 1 1]


# Train Model

In [40]:
from sklearn.model_selection import train_test_split

test_split = 0.2

#trainX, testX, trainY, testY = train_test_split(X, Y_no_empty_classes, test_size=test_split, random_state=1)
#Do not giva a value for random_state - this way random will be different every time
trainX, testX, trainY, testY = train_test_split(X, Y_label, test_size=test_split)

print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(102400, 4) (25600, 4) (102400,) (25600,)


In [41]:
##Model
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(trainX, trainY)
predictions = logisticRegr.predict(testX)

In [42]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [43]:
#Classification Report
print(classification_report(testY, predictions))

              precision    recall  f1-score   support

           1       0.95      0.95      0.95     12748
           2       0.95      0.95      0.95     12852

    accuracy                           0.95     25600
   macro avg       0.95      0.95      0.95     25600
weighted avg       0.95      0.95      0.95     25600

